In [1]:
import paddle.fluid as fluid
import paddle
import numpy as np
from paddle.fluid.debugger import draw_block_graphviz

In [2]:
np.random.seed(0)
X_train = np.random.random(size=(4, 3, 10, 10)).astype('float32')
Y_train = np.array([[1], [1], [1], [1]]).astype('float32')

In [3]:
def net(image):
    y = fluid.layers.conv2d(name="conv2d", input=image, num_filters=3, filter_size=3, act='relu')
    y = fluid.layers.Print(y, summarize=10, message="Conv2d")
    pool2d = fluid.layers.pool2d(name="pool2d", input=y, pool_size=y.shape[2:])
    pool2d = fluid.layers.Print(pool2d, summarize=10, message="Pool2d")
    y = fluid.layers.fc(name='predict', input=pool2d, size=1)
    return (y, pool2d)

## Trainer & Inferencer

In [4]:
def reader():
    l = X_train.shape[0]
    for i in range(l):
        yield X_train[i], Y_train[i]

train_reader = paddle.batch(reader, 2)

In [5]:
def optimizer_program():
    return fluid.optimizer.Adam(learning_rate=0.001)

In [6]:
def train_program():
    label = fluid.layers.data(name='label', shape=[1], dtype='float32')
    image = fluid.layers.data(name='image', shape=[3, 10, 10], dtype='float32')
    predict, _ = net(image)
    cost = fluid.layers.square_error_cost(input=predict, label=label)
    avg_cost = fluid.layers.mean(cost)
    return [avg_cost]

In [7]:
def infer_program():
    image = fluid.layers.data(name='image', shape=[3, 10, 10], dtype='float32')
    predict, _= net(image)
    return predict

In [8]:
trainer = fluid.Trainer(
    train_func=train_program, place=fluid.CPUPlace(), optimizer_func=optimizer_program)

In [9]:
def event_handler(event):
   if isinstance(event, fluid.EndEpochEvent):
        trainer.save_params("./model")

In [ ]:
trainer.train(
    num_epochs=200,
    event_handler=event_handler,
    reader=train_reader,
    feed_order=["image", "label"]
)

In [ ]:
!ls model

In [ ]:
infer = fluid.Inferencer(infer_func=infer_program, param_path="model", place=fluid.CPUPlace())

In [ ]:
infer.infer({'image': X_train})

In [ ]:
with fluid.executor.scope_guard(infer.scope):
    result = infer.exe.run(infer.inference_program, fetch_list=[infer.predict_var], feed={'image': X_train})
result

In [ ]:
# Draw a graph to find variable names
draw_block_graphviz(infer.inference_program.blocks[0], path="g.pdf")

In [ ]:
# fetch param and varialbe by name
infer.exe.run(infer.inference_program, 
              fetch_list=[infer.predict_var, 'conv2d.b_0', 'pool2d.tmp_0'],  
              feed={'image': X_train},
              scope=infer.scope)

In [ ]:
b = fluid.global_scope().find_var('conv2d.b_0') # Trainer & Inferencer runs in a new scope
b, type(b)

## Executor

In [ ]:
label = fluid.layers.data(name='label', shape=[1], dtype='float32')
image = fluid.layers.data(name='image', shape=[3, 10, 10], dtype='float32')
[predict, pool2d] = net(image)
inference_program = fluid.default_main_program().clone(for_test=True)

In [ ]:
cost = fluid.layers.square_error_cost(input=predict, label=label)
avg_cost = fluid.layers.mean(cost)

In [ ]:
optimizer = fluid.optimizer.Adam(learning_rate=0.01)

In [ ]:
_ = optimizer.minimize(avg_cost)

In [ ]:
exe = fluid.Executor(fluid.CPUPlace())

In [ ]:
exe.run(fluid.default_startup_program())

In [ ]:
feed={'image': X_train, 'label': Y_train}

In [ ]:
for i in range(200):
    cost, _ = exe.run(fetch_list=[avg_cost, pool2d], feed=feed)
    if i % 50 == 0:
        print("Epoch %i, cost %f" % (i, cost))
print("Epoch %i, cost %f" % (i, cost))

In [ ]:
fluid.io.save_params(exe, dirname="exe-model")
!ls exe-model

In [ ]:
exe.run(inference_program, fetch_list=[predict], feed=feed)

In [ ]:
fluid.io.load_params(exe, dirname="exe-model")